<a href="https://colab.research.google.com/github/boseongkang/mlproject/blob/main/bilboard_song.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/cs/cs171/project

/content/drive/MyDrive/cs/cs171/project


In [3]:
pwd

'/content/drive/MyDrive/cs/cs171/project'

## Use TPU (colab pro)

## Use GPU

In [ ]:
!nvidia-smi

In [4]:
!pip install billboard.py

In [120]:
import billboard as b
import pandas as pd
import os
import numpy as np
from datetime import datetime
from collections import Counter
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
# chart = b.ChartData('hot-100')
chart = b.ChartData('pop-songs', date='2000-01-01')
chart.title

2000 01 01
2000 02 01
2000 03 01
2000 04 01
2000 05 01
2000 06 01
2000 07 01
2000 08 01
2000 09 01
2000 10 01
2000 11 01
2000 12 01


'Pop Airplay'

In [ ]:
song_lib = []

for song in chart:
    song_lib.append({
        'rank': song.rank,
        'title': song.title,
        'artist': song.artist
    })
song_lib

[{'rank': 1, 'title': 'Smooth', 'artist': 'Santana Featuring Rob Thomas'},
 {'rank': 2, 'title': 'I Knew I Loved You', 'artist': 'Savage Garden'},
 {'rank': 3, 'title': 'Back At One', 'artist': 'Brian McKnight'},
 {'rank': 4, 'title': 'What A Girl Wants', 'artist': 'Christina Aguilera'},
 {'rank': 5, 'title': 'I Need To Know', 'artist': 'Marc Anthony'},
 {'rank': 6, 'title': 'Blue (Da Ba Dee)', 'artist': 'Eiffel 65'},
 {'rank': 7, 'title': 'Then The Morning Comes', 'artist': 'Smash Mouth'},
 {'rank': 8, 'title': 'Waiting For Tonight', 'artist': 'Jennifer Lopez'},
 {'rank': 9, 'title': 'Bring It All To Me', 'artist': 'Blaque'},
 {'rank': 10, 'title': 'Steal My Sunshine', 'artist': 'Len'},
 {'rank': 11, 'title': 'Meet Virginia', 'artist': 'Train'},
 {'rank': 12, 'title': 'Shake Your Bon-Bon', 'artist': 'Ricky Martin'},
 {'rank': 13, 'title': 'My Love Is Your Love', 'artist': 'Whitney Houston'},
 {'rank': 14,
  'title': 'I Wanna Love You Forever',
  'artist': 'Jessica Simpson'},
 {'rank':

lyrics download

In [6]:
import kagglehub
kaggle_data = kagglehub.dataset_download("suparnabiswas/billboard-hot-1002000-2023-data-with-features")

100%|██████████| 1.93M/1.93M [00:00<00:00, 3.24MB/s]

Extracting files...


In [7]:
folder_path = "/content/drive/MyDrive/cs/cs171/project"
!cp {kaggle_data}/* {folder_path}

In [9]:
df = pd.read_csv('billboard_24years_lyrics_spotify.csv')
df.head()

,ranking,song,band_singer,songurl,titletext,url,year,lyrics,uri,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,1,Breathe,Faith Hill,/wiki/Breathe_(Faith_Hill_song),Breathe,/wiki/Faith_Hill,2000,I can feel the magic floating in the air\nBein...,spotify:track:3y4LxiYMgDl4RethdzpmNe,0.529,...,0.000000,0.2510,0.278,136.859,audio_features,3y4LxiYMgDl4RethdzpmNe,https://api.spotify.com/v1/tracks/3y4LxiYMgDl4...,https://api.spotify.com/v1/audio-analysis/3y4L...,250547.0,4.0
1,2,Smooth,Santana,/wiki/Smooth_(Santana_song),Smooth,/wiki/Santana_(band),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,...,0.000005,0.2950,0.961,115.996,audio_features,0n2SEXB2qoRQg171q7XqeW,https://api.spotify.com/v1/tracks/0n2SEXB2qoRQ...,https://api.spotify.com/v1/audio-analysis/0n2S...,294987.0,4.0
2,2,Smooth,Rob Thomas,/wiki/Smooth_(Santana_song),Smooth,/wiki/Rob_Thomas_(musician),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:5IALWUYK0zDSEmZgb4ICvc,0.590,...,0.807000,0.2990,0.724,115.983,audio_features,5IALWUYK0zDSEmZgb4ICvc,https://api.spotify.com/v1/tracks/5IALWUYK0zDS...,https://api.spotify.com/v1/audio-analysis/5IAL...,244924.0,4.0
3,3,Maria Maria,Santana,/wiki/Maria_Maria,Maria Maria,/wiki/Santana_(band),2000,"Ladies and gents, turn up your sound systems\n...",spotify:track:3XKIUb7HzIF1Vu9usunMzc,0.777,...,0.002010,0.0348,0.680,97.911,audio_features,3XKIUb7HzIF1Vu9usunMzc,https://api.spotify.com/v1/tracks/3XKIUb7HzIF1...,https://api.spotify.com/v1/audio-analysis/3XKI...,261973.0,4.0
4,3,Maria Maria,The Product G&B,/wiki/Maria_Maria,Maria Maria,/wiki/The_Product_G%26B,2000,Turn up this sound system\nTo the sound of Car...,spotify:track:3XKIUb7HzIF1Vu9usunMzc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# df['year'].nunique()
df['year'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022, 2023])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ranking           3397 non-null   int64  
 1   song              3397 non-null   object 
 2   band_singer       3397 non-null   object 
 3   songurl           3397 non-null   object 
 4   titletext         3397 non-null   object 
 5   url               3397 non-null   object 
 6   year              3397 non-null   int64  
 7   lyrics            3397 non-null   object 
 8   uri               3397 non-null   object 
 9   danceability      486 non-null    float64
 10  energy            486 non-null    float64
 11  key               486 non-null    float64
 12  loudness          486 non-null    float64
 13  mode              486 non-null    float64
 14  speechiness       486 non-null    float64
 15  acousticness      486 non-null    float64
 16  instrumentalness  486 non-null    float64


In [13]:
def get_monthly_billboard_charts(start_year, end_year, chart_name = 'hot-100'):
    billboard_data = []

    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            date_str = f'{year}-{month:02d}-01'

            try:
                chart = b.ChartData(chart_name, date=date_str)

                for song in chart:
                    billboard_data.append({
                        'date': date_str,
                        'year': year,
                        'month': month,
                        'rank': song.rank,
                        'title': song.title,
                        'artist': song.artist,
                        'weeks': song.weeks
                    })

                print(f'{date_str} collecting')

            except Exception as e:
                print(f'{date_str} fail: {e}')

    df_billboard = pd.DataFrame(billboard_data)
    return df_billboard

In [ ]:
# pwd
# !ls

'bilboard song.ipynb'			   contraction_word.gsheet
 bilboardTop100.ipynb			  'test (1).ipynb'
 billboard_24years_lyrics_spotify.csv	   word_breaker.ipynb
 billboard_24years_lyrics_spotify.gsheet


In [16]:
def match_billboard_with_csv(billboard_df, csv_path, csv_title_colum = 'titletext'):
    df_csv = pd.read_csv(csv_path)

    billboard_df['title_normalized'] = billboard_df['title'].str.lower().str.strip()
    df_csv['title_normalized'] = df_csv[csv_title_column].str.lower().str.strip()

    csv_titles = set(df_csv['title_normalized'].unique())

    billboard_df['matched'] = billboard_df['title_normalized'].isin(csv_titles)

    matched_df = billboard_df[billboard_df['matched'] == True].copy()

    result_df = matched_df.merge(
        df_csv[['title_normalized', 'lyrics', 'url']],
        on='title_normalized',
        how='left'
    )

    print(f'\ntotal song: {len(billboard_df)}')
    print(f'match with csv file: {len(matched_df)}')
    print(f'match rate: {len(matched_df)/len(billboard_df)*100:.2f}%')

    return result_df

In [111]:
# run only one time to save songs
# if __name__ == "__main__":
#     df_billboard = get_monthly_billboard_charts(2000, 2023, chart_name = 'hot-100')

#     csv_file_path = 'billboard_24years_lyrics_spotify.csv'
#     df_matched = match_billboard_with_csv(df_billboard, csv_file_path, csv_title_column = 'titletext')

#     df_billboard.to_csv('billboard_2000_2023_monthly.csv', index = False, encoding = 'utf-8')
#     df_matched.to_csv('billboard_matched_with_csv.csv', index = False, encoding = 'utf-8')

#     print("\nmatched data")
#     print(df_matched.head(10))

#     print("\nmatch data by year")
#     yearly_stats = df_matched.groupby('year').agg({
#         'title': 'count',
#         'rank': 'mean'
#     }).rename(columns={'title': 'matched songs', 'rank': 'rank'})
#     print(yearly_stats)


took 1 hr and 40 minute

In [ ]:
# billboard

In [ ]:
df_billboard.to_csv('monthly_song_2000~2023.csv', index=False, encoding='utf-8')
print(f"save sucess{len(df_billboard):,} data")

save sucess28,800 data


In [ ]:
!ls -l

total 11087
-rw------- 1 root root   61560 Nov 23 22:09  bilboard_song.ipynb
-rw------- 1 root root   75680 Nov 12 00:12  bilboardTop100.ipynb
-rw------- 1 root root 9449053 Nov 12 00:05  billboard_24years_lyrics_spotify.csv
-rw------- 1 root root     176 Nov 12 00:04  billboard_24years_lyrics_spotify.gsheet
-rw------- 1 root root     176 Nov 12 00:13  contraction_word.gsheet
-rw------- 1 root root 1614328 Nov 23 22:07  monthly_song_2000~2023.csv
-rw------- 1 root root   85047 Nov 14 22:33 'test (1).ipynb'
-rw------- 1 root root   64741 Nov 12 00:13  word_breaker.ipynb


In [ ]:
def match_billboard_titles(billboard_csv, kaggle_csv, billboard_title_col='title', kaggle_title_col='titletext'):
    df_billboard = pd.read_csv(billboard_csv)
    df_kaggle = pd.read_csv(kaggle_csv)

    df_billboard['title_norm'] = df_billboard[billboard_title_col].str.lower().str.strip()
    df_kaggle['titletext_norm'] = df_kaggle[kaggle_title_col].str.lower().str.strip()

    kaggle_titles_set = set(df_kaggle['titletext_norm'].unique())
    df_billboard['matched'] = df_billboard['title_norm'].isin(kaggle_titles_set)

    matched_df = df_billboard[df_billboard['matched'] == True].copy()

    print('total songs: ', len(df_billboard))
    print('matched songs: ', len(matched_df))
    print('matching rate: ', len(matched_df)/len(df_billboard)*100)

    return df_billboard, matched_df

billboard_path = 'monthly_song_2000~2023.csv'
kaggle_path = 'billboard_24years_lyrics_spotify.csv'

df_all, df_matched = match_billboard_titles(billboard_path, kaggle_path)

df_all.to_csv('billboard_with_matched.csv', index=False)
df_matched.to_csv('billboard_matched_only.csv', index=False)

In [26]:
ls

 audio_analysis.ipynb                      billboard_with_matched.csv
 bilboard_song.ipynb                       contraction_word.gsheet
 bilboardTop100.ipynb                      monthly_song_2000~2023.csv
 billboard_24years_lyrics_spotify.csv     'test (1).ipynb'
 billboard_24years_lyrics_spotify.gsheet   word_breaker.ipynb
 billboard_matched_only.csv


In [54]:
billboard_path = 'monthly_song_2000~2023.csv'
kaggle_path = 'billboard_24years_lyrics_spotify.csv'

In [55]:
billboard = pd.read_csv(billboard_path)
kaggle = pd.read_csv(kaggle_path)
billboard
# kaggle

,date,year,month,rank,title,artist,weeks
0,2000-01-01,2000,1,1,Smooth,Santana Featuring Rob Thomas,23
1,2000-01-01,2000,1,2,Back At One,Brian McKnight,19
2,2000-01-01,2000,1,3,I Wanna Love You Forever,Jessica Simpson,12
3,2000-01-01,2000,1,4,My Love Is Your Love,Whitney Houston,18
4,2000-01-01,2000,1,5,I Knew I Loved You,Savage Garden,11
...,...,...,...,...,...,...,...
28795,2023-12-01,2023,12,96,Different 'Round Here,Riley Green Featuring Luke Combs,2
28796,2023-12-01,2023,12,97,Bongos,Cardi B & Megan Thee Stallion,11
28797,2023-12-01,2023,12,98,But I Got A Beer In My Hand,Luke Bryan,6
28798,2023-12-01,2023,12,99,Calling For You,Drake Featuring 21 Savage,4


In [56]:
billboard['title_norm'] = billboard['title'].str.lower().str.strip()
kaggle['title_norm'] = kaggle['titletext'].str.lower().str.strip()
# kaggle

In [57]:
billboard['matched'] = billboard['title_norm'].isin(set(kaggle['title_norm']))
matched = billboard[billboard['matched']]
unique_bb = billboard['title_norm'].nunique()
unique_mt = matched['title_norm'].nunique()
total = len(billboard)
matched_total = len(matched)
rate = matched_total / total * 100
unique_rate = unique_mt / unique_bb * 100


print(f"Total rows: {total}")
print(f"Matched rows: {matched_total}")
print(f"Matching rate: {rate:.2f}%")
print(f"Unique Billboard titles: {unique_bb}")
print(f"Unique matched titles: {unique_mt}")
print(f"Unique title matching rate: {unique_rate:.2f}%")

Total rows: 28800
Matched rows: 14207
Matching rate: 49.33%
Unique Billboard titles: 7089
Unique matched titles: 2015
Unique title matching rate: 28.42%


In [58]:
title_counts = Counter(billboard['title_norm'])
duplicates = [title for title, count in title_counts.items() if count > 1]

print(f"Number of duplicate titles: {len(duplicates)}")
print("\nSample duplicated titles:\n")
for t in duplicates[:20]:
    print(t)

Number of duplicate titles: 5460

Sample duplicated titles:

smooth
back at one
i wanna love you forever
my love is your love
i knew i loved you
i need to know
hot boyz
u know what's up
bring it all to me
girl on tv
what a girl wants
24/7
blue (da ba dee)
then the morning comes
he can't love u
waiting for tonight
that's the way it is
dancin'
get it on tonite
learn to fly


In [59]:
billboard

,date,year,month,rank,title,artist,weeks,title_norm,matched
0,2000-01-01,2000,1,1,Smooth,Santana Featuring Rob Thomas,23,smooth,True
1,2000-01-01,2000,1,2,Back At One,Brian McKnight,19,back at one,True
2,2000-01-01,2000,1,3,I Wanna Love You Forever,Jessica Simpson,12,i wanna love you forever,True
3,2000-01-01,2000,1,4,My Love Is Your Love,Whitney Houston,18,my love is your love,True
4,2000-01-01,2000,1,5,I Knew I Loved You,Savage Garden,11,i knew i loved you,True
...,...,...,...,...,...,...,...,...,...
28795,2023-12-01,2023,12,96,Different 'Round Here,Riley Green Featuring Luke Combs,2,different 'round here,False
28796,2023-12-01,2023,12,97,Bongos,Cardi B & Megan Thee Stallion,11,bongos,False
28797,2023-12-01,2023,12,98,But I Got A Beer In My Hand,Luke Bryan,6,but i got a beer in my hand,False
28798,2023-12-01,2023,12,99,Calling For You,Drake Featuring 21 Savage,4,calling for you,False


In [60]:
def lyrics_to_wordlist(lyrics_series):
    words = []
    for text in lyrics_series.dropna():
        words.extend(text.split())
    return words

word_list = lyrics_to_wordlist(kaggle['lyrics'])
len(word_list)

1737591

In [66]:
kaggle.head()

,ranking,song,band_singer,songurl,titletext,url,year,lyrics,uri,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,title_norm
0,1,Breathe,Faith Hill,/wiki/Breathe_(Faith_Hill_song),Breathe,/wiki/Faith_Hill,2000,I can feel the magic floating in the air\nBein...,spotify:track:3y4LxiYMgDl4RethdzpmNe,0.529,...,0.2510,0.278,136.859,audio_features,3y4LxiYMgDl4RethdzpmNe,https://api.spotify.com/v1/tracks/3y4LxiYMgDl4...,https://api.spotify.com/v1/audio-analysis/3y4L...,250547.0,4.0,breathe
1,2,Smooth,Santana,/wiki/Smooth_(Santana_song),Smooth,/wiki/Santana_(band),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,...,0.2950,0.961,115.996,audio_features,0n2SEXB2qoRQg171q7XqeW,https://api.spotify.com/v1/tracks/0n2SEXB2qoRQ...,https://api.spotify.com/v1/audio-analysis/0n2S...,294987.0,4.0,smooth
2,2,Smooth,Rob Thomas,/wiki/Smooth_(Santana_song),Smooth,/wiki/Rob_Thomas_(musician),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:5IALWUYK0zDSEmZgb4ICvc,0.590,...,0.2990,0.724,115.983,audio_features,5IALWUYK0zDSEmZgb4ICvc,https://api.spotify.com/v1/tracks/5IALWUYK0zDS...,https://api.spotify.com/v1/audio-analysis/5IAL...,244924.0,4.0,smooth
3,3,Maria Maria,Santana,/wiki/Maria_Maria,Maria Maria,/wiki/Santana_(band),2000,"Ladies and gents, turn up your sound systems\n...",spotify:track:3XKIUb7HzIF1Vu9usunMzc,0.777,...,0.0348,0.680,97.911,audio_features,3XKIUb7HzIF1Vu9usunMzc,https://api.spotify.com/v1/tracks/3XKIUb7HzIF1...,https://api.spotify.com/v1/audio-analysis/3XKI...,261973.0,4.0,maria maria
4,3,Maria Maria,The Product G&B,/wiki/Maria_Maria,Maria Maria,/wiki/The_Product_G%26B,2000,Turn up this sound system\nTo the sound of Car...,spotify:track:3XKIUb7HzIF1Vu9usunMzc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,maria maria


In [106]:
billboard[billboard['title'] == 'Smooth']

,date,year,month,rank,title,artist,weeks,title_norm,matched
0,2000-01-01,2000,1,1,Smooth,Santana Featuring Rob Thomas,23,smooth,True
102,2000-02-01,2000,2,3,Smooth,Santana Featuring Rob Thomas,28,smooth,True
212,2000-03-01,2000,3,13,Smooth,Santana Featuring Rob Thomas,32,smooth,True
307,2000-04-01,2000,4,8,Smooth,Santana Featuring Rob Thomas,36,smooth,True
415,2000-05-01,2000,5,16,Smooth,Santana Featuring Rob Thomas,41,smooth,True
522,2000-06-01,2000,6,23,Smooth,Santana Featuring Rob Thomas,45,smooth,True
631,2000-07-01,2000,7,32,Smooth,Santana Featuring Rob Thomas,49,smooth,True
738,2000-08-01,2000,8,39,Smooth,Santana Featuring Rob Thomas,54,smooth,True
847,2000-09-01,2000,9,48,Smooth,Santana Featuring Rob Thomas,58,smooth,True
21491,2017-11-01,2017,11,92,Smooth,Florida Georgia Line,3,smooth,True


In [89]:
word_count_dict = {}

for idx, row in kaggle.iterrows():
    title = row['title_norm']
    lyrics = row['lyrics']
    if pd.isnull(lyrics):
        continue
    words = str(lyrics).split()
    if title not in word_count_dict:
        word_count_dict[title] = Counter()
    word_count_dict[title].update(words)
# word_count_dict

In [90]:
top_titles = list(word_count_dict.keys())[:5]

for title in top_titles:
    print(title)
    print(word_count_dict[title].most_common(20))

breathe
[('the', 14), ('I', 11), ('you', 10), ('that', 8), ('to', 8), ('can', 7), ('feel', 7), ('in', 6), ('breathe', 6), ('and', 5), ('up', 5), ('And', 5), ('Baby,', 5), ('me', 4), ('way', 4), ('just', 4), ("I'm", 4), ('your', 3), ('The', 3), ("There's", 3)]
smooth
[('the', 44), ('you', 26), ('forget', 22), ('about', 22), ('I', 18), ('my', 16), ('it', 16), ("it's", 14), ('so', 14), ('your', 14), ('it,', 14), ('no,', 14), ('that', 12), ('else', 12), ('from', 10), ('Well,', 10), ('and', 10), ('You', 10), ("don't", 10), ('yeah', 8)]
maria maria
[('the', 46), ('mama', 36), ('Ahora', 24), ('vengo', 24), ('of', 21), ('Maria', 20), ('in', 20), ('chola', 18), ('a', 15), ('up', 14), ('She', 14), ('chula', 12), ('Carlos', 10), ('Santana', 10), ('like', 9), ('To', 8), ('Oh,', 8), ('Maria,', 8), ('me', 8), ('just', 7)]
i wanna know
[('you', 32), ('I', 32), ('wanna', 17), ('to', 17), ('what', 14), ('know', 12), ('like', 12), ('So', 10), ('can', 10), ('the', 10), ('be', 9), ('that', 9), ('do', 9), 

In [67]:
single_word_count_dict = {}
words = lyrics_to_wordlist(kaggle['lyrics'])
grouped = df.drop_duplicates(subset=['titletext', 'lyrics']).groupby('titletext')

for title, group in grouped:
    lyrics = group['lyrics'].dropna().iloc[0]
    single_word_count_dict[title] = Counter(words)

print(single_word_count_dict['Smooth'].most_common(20))

[('I', 61129), ('the', 49151), ('you', 48540), ('a', 28937), ('to', 27430), ('me', 25408), ('it', 23632), ('my', 22243), ("I'm", 18810), ('and', 18734), ('in', 17574), ('like', 16998), ('that', 15648), ('on', 15393), ('your', 13628), ('And', 11711), ('You', 11279), ('of', 10637), ('be', 10629), ('know', 10420)]


In [87]:
# first_key = list(word_count_dict.keys())[0]
# # first_key
# word_count_dict[first_key]
# # word_count_dict

In [86]:
# df['titletext'].unique().tolist()

In [ ]:
a = df[df['titletext'] == 'New Divide']['lyrics'].tolist()
# print(a)
for text in a:
    print(text)

I remembered black skies
The lightning all around me
I remembered each flash as time began to blur
Like a startling sign, that fate had finally found me

And your voice was all I heard
That I get what I deserve

So give me reason to prove me wrong
To wash this memory clean
Let the floods cross the distance in your eyes
Give me reason to fill this hole, connect this space between
Let it be enough to reach the truth that lies
Across this new divide

There was nothing in sight but memories left abandoned
There was nowhere to hide
The ashes fell like snow
And the ground caved in between where we were standing

And your voice was all I heard
That I get what I deserve
You might also like
So give me reason to prove me wrong
To wash this memory clean
Let the floods cross the distance in your eyes
Across this new divide

In every loss, in every lie
In every truth that you deny
And each regret and each goodbye was a mistake too great to hide

And your voice was all I heard
That I get what I dese

In [83]:
merged = billboard.merge(
    kaggle[['title_norm', 'lyrics']],
    on = 'title_norm',
    how = 'left'
)
merged

,date,year,month,rank,title,artist,weeks,title_norm,matched,lyrics
0,2000-01-01,2000,1,1,Smooth,Santana Featuring Rob Thomas,23,smooth,True,"Man, it's a hot one\nLike seven inches from th..."
1,2000-01-01,2000,1,1,Smooth,Santana Featuring Rob Thomas,23,smooth,True,"Man, it's a hot one\nLike seven inches from th..."
2,2000-01-01,2000,1,2,Back At One,Brian McKnight,19,back at one,True,It's undeniable\nThat we should be together\nI...
3,2000-01-01,2000,1,3,I Wanna Love You Forever,Jessica Simpson,12,i wanna love you forever,True,"Ooh, ooh\nOoh, ah\n\nYou set my soul at ease\n..."
4,2000-01-01,2000,1,4,My Love Is Your Love,Whitney Houston,18,my love is your love,True,"Clap your hands, y'all, it's alright\n(Turn me..."
...,...,...,...,...,...,...,...,...,...,...
40288,2023-12-01,2023,12,96,Different 'Round Here,Riley Green Featuring Luke Combs,2,different 'round here,False,NaN
40289,2023-12-01,2023,12,97,Bongos,Cardi B & Megan Thee Stallion,11,bongos,False,NaN
40290,2023-12-01,2023,12,98,But I Got A Beer In My Hand,Luke Bryan,6,but i got a beer in my hand,False,NaN
40291,2023-12-01,2023,12,99,Calling For You,Drake Featuring 21 Savage,4,calling for you,False,NaN


In [84]:
song_level = merged.dropna(subset=['lyrics']).drop_duplicates(
    subset=['title_norm', 'lyrics']
)
song_level

,date,year,month,rank,title,artist,weeks,title_norm,matched,lyrics
0,2000-01-01,2000,1,1,Smooth,Santana Featuring Rob Thomas,23,smooth,True,"Man, it's a hot one\nLike seven inches from th..."
2,2000-01-01,2000,1,2,Back At One,Brian McKnight,19,back at one,True,It's undeniable\nThat we should be together\nI...
3,2000-01-01,2000,1,3,I Wanna Love You Forever,Jessica Simpson,12,i wanna love you forever,True,"Ooh, ooh\nOoh, ah\n\nYou set my soul at ease\n..."
4,2000-01-01,2000,1,4,My Love Is Your Love,Whitney Houston,18,my love is your love,True,"Clap your hands, y'all, it's alright\n(Turn me..."
5,2000-01-01,2000,1,5,I Knew I Loved You,Savage Garden,11,i knew i loved you,True,"Mmm\nOoh, oh\n\nMaybe it's intuition\nBut some..."
...,...,...,...,...,...,...,...,...,...,...
39779,2023-08-01,2023,8,61,Watermelon Moonshine,Lainey Wilson,5,watermelon moonshine,True,It was right after senior year\nJust before th...
39823,2023-09-01,2023,9,1,Rich Men North Of Richmond,Oliver Anthony Music,2,rich men north of richmond,True,"I've been sellin' my soul, workin' all day\nOv..."
39829,2023-09-01,2023,9,5,Paint The Town Red,Doja Cat,3,paint the town red,True,By\nWalk on by\nWalk on by\nWalk on by\nWalk o...
39852,2023-09-01,2023,9,22,Meltdown,Travis Scott Featuring Drake,4,meltdown,True,\nYeah\nTensions is definitely rising\nT'd up ...


In [103]:
song_word_counts = {}

for _, row in song_level.iterrows():
    title = row['title_norm']
    lyrics = str(row['lyrics'])
    words = lyrics.split()
    song_word_counts[title] = Counter(words)

print(song_word_counts['new divide'])

Counter({'this': 12, 'I': 11, 'me': 10, 'to': 10, 'the': 9, 'your': 6, 'was': 6, 'in': 6, 'And': 5, 'reason': 5, 'Let': 5, 'Across': 5, 'new': 5, 'all': 4, 'that': 4, 'divide': 4, 'each': 3, 'voice': 3, 'heard': 3, 'That': 3, 'get': 3, 'what': 3, 'deserve': 3, 'So': 3, 'give': 3, 'prove': 3, 'wrong': 3, 'To': 3, 'wash': 3, 'memory': 3, 'clean': 3, 'floods': 3, 'cross': 3, 'distance': 3, 'eyes': 3, 'between': 3, 'truth': 3, 'every': 3, 'remembered': 2, 'The': 2, 'a': 2, 'Give': 2, 'fill': 2, 'hole,': 2, 'connect': 2, 'space': 2, 'it': 2, 'be': 2, 'enough': 2, 'reach': 2, 'lies': 2, 'There': 2, 'hide': 2, 'like': 2, 'In': 2, 'black': 1, 'skies': 1, 'lightning': 1, 'around': 1, 'flash': 1, 'as': 1, 'time': 1, 'began': 1, 'blur': 1, 'Like': 1, 'startling': 1, 'sign,': 1, 'fate': 1, 'had': 1, 'finally': 1, 'found': 1, 'nothing': 1, 'sight': 1, 'but': 1, 'memories': 1, 'left': 1, 'abandoned': 1, 'nowhere': 1, 'ashes': 1, 'fell': 1, 'snow': 1, 'ground': 1, 'caved': 1, 'where': 1, 'we': 1, 'we

In [115]:
norm = lambda s: str(s).replace("’","'").replace("`","'")
token_pattern = re.compile(r"\b[A-Za-z]+'[A-Za-z]+\b|\b[A-Za-z]+\b")

def tokenize(text):
    text = norm(text)
    return [w.lower() for w in token_pattern.findall(text)]

all_counter = Counter()

for t in merged['lyrics'].dropna().astype(str):
    all_counter.update(tokenize(t))

top_words = all_counter.most_common(10)
top_words

[('you', 500630),
 ('i', 500618),
 ('the', 380780),
 ('me', 239638),
 ('a', 227774),
 ('and', 225944),
 ('it', 219610),
 ('to', 216000),
 ('my', 181603),
 ('oh', 152835)]

In [117]:
def month_to_season(m):
    if m in [12, 1, 2]:
        return "winter"
    elif m in [3, 4, 5]:
        return "spring"
    elif m in [6, 7, 8]:
        return "summer"
    else:
        return "fall"

billboard["season"] = billboard["month"].apply(month_to_season)
merged["season"] = merged["month"].apply(month_to_season)
billboard

,date,year,month,rank,title,artist,weeks,title_norm,matched,season
0,2000-01-01,2000,1,1,Smooth,Santana Featuring Rob Thomas,23,smooth,True,winter
1,2000-01-01,2000,1,2,Back At One,Brian McKnight,19,back at one,True,winter
2,2000-01-01,2000,1,3,I Wanna Love You Forever,Jessica Simpson,12,i wanna love you forever,True,winter
3,2000-01-01,2000,1,4,My Love Is Your Love,Whitney Houston,18,my love is your love,True,winter
4,2000-01-01,2000,1,5,I Knew I Loved You,Savage Garden,11,i knew i loved you,True,winter
...,...,...,...,...,...,...,...,...,...,...
28795,2023-12-01,2023,12,96,Different 'Round Here,Riley Green Featuring Luke Combs,2,different 'round here,False,winter
28796,2023-12-01,2023,12,97,Bongos,Cardi B & Megan Thee Stallion,11,bongos,False,winter
28797,2023-12-01,2023,12,98,But I Got A Beer In My Hand,Luke Bryan,6,but i got a beer in my hand,False,winter
28798,2023-12-01,2023,12,99,Calling For You,Drake Featuring 21 Savage,4,calling for you,False,winter


In [118]:
season_counter = Counter()
for _, row in merged.dropna(subset=['lyrics']).iterrows():
    words = tokenize(row['lyrics'])
    season_counter.update(words)

spring_counter = Counter()
for _, row in merged[merged['season']=='spring'].dropna(subset=['lyrics']).iterrows():
    spring_counter.update(tokenize(row['lyrics']))
spring_counter.most_common(50)


[('you', 131016),
 ('i', 130164),
 ('the', 98765),
 ('me', 61873),
 ('a', 59034),
 ('it', 58732),
 ('and', 58704),
 ('to', 56211),
 ('my', 49096),
 ('oh', 39424),
 ("i'm", 39164),
 ('like', 38453),
 ('in', 36644),
 ('that', 35410),
 ('on', 33365),
 ('yeah', 31762),
 ('your', 27548),
 ('up', 24795),
 ('know', 23346),
 ("don't", 23319),
 ('we', 22356),
 ('be', 22087),
 ('all', 22069),
 ('so', 21231),
 ('with', 21132),
 ('of', 20116),
 ('got', 19127),
 ('no', 19048),
 ('for', 18659),
 ('just', 18534),
 ('but', 18205),
 ('baby', 17939),
 ('get', 17262),
 ('love', 17187),
 ('this', 16874),
 ('is', 16399),
 ('when', 16240),
 ("it's", 15517),
 ('what', 15283),
 ('do', 14821),
 ('go', 14792),
 ('she', 14428),
 ('ooh', 14167),
 ('now', 13960),
 ('if', 13822),
 ('can', 12400),
 ('down', 12180),
 ("you're", 12087),
 ("ain't", 11833),
 ('wanna', 11680)]

In [121]:
song_season = merged.dropna(subset=['lyrics']).copy()
song_season['season'] = song_season['month'].map(
    {12:'winter',1:'winter',2:'winter',
     3:'spring',4:'spring',5:'spring',
     6:'summer',7:'summer',8:'summer',
     9:'fall',10:'fall',11:'fall'}
)

X = song_season['lyrics'].astype(str)
y = song_season['season']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = Pipeline([
    ("tfidf", TfidfVectorizer(
        tokenizer=tokenize,
        min_df=10,
        max_df=0.7,
        ngram_range=(1,2)
    )),
    ("clf", LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)
print("Accuracy:", model.score(X_test, y_test))

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy: 0.2926070038910506


In [126]:
merged['hit'] = (merged['weeks'] >= 20).astype(int)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

X = merged['lyrics'].astype(str)
y = merged['hit']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = Pipeline([
    ("tfidf", TfidfVectorizer(tokenizer=tokenize,
                              min_df=10,
                              max_df=0.7,
                              ngram_range=(1,2))),
    ("clf", LogisticRegression(max_iter=2000))
])

model.fit(X_train, y_train)
print("Accuracy:", model.score(X_test, y_test))

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy: 0.7925300905819581


In [128]:
merged['is_top10'] = (merged['rank'] <= 10).astype(int)
X = merged['lyrics'].astype(str)
y = merged['is_top10']


In [129]:
tfidf = model.named_steps['tfidf']
clf = model.named_steps['clf']

feature_names = tfidf.get_feature_names_out()
coefs = clf.coef_[0]

top_pos_idx = coefs.argsort()[-30:][::-1]
top_neg_idx = coefs.argsort()[:30]

top_pos_words = [(feature_names[i], coefs[i]) for i in top_pos_idx]
top_neg_words = [(feature_names[i], coefs[i]) for i in top_neg_idx]


In [131]:
import re
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


data = merged.dropna(subset=['lyrics']).copy()
data['is_top10'] = (data['rank'] <= 10).astype(int)

X = data['lyrics'].astype(str)
y = data['is_top10']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = Pipeline([
    ("tfidf", TfidfVectorizer(
        tokenizer=tokenize,
        min_df=10,
        max_df=0.7,
        ngram_range=(1,2)
    )),
    ("clf", LogisticRegression(
        max_iter=2000,
        class_weight="balanced"
    ))
])

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:,1]

print("Accuracy:", model.score(X_test, y_test))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

scores = cross_val_score(model, X, y, cv=10, scoring='roc_auc')
print("CV ROC-AUC mean:", scores.mean(), "std:", scores.std())

tfidf = model.named_steps['tfidf']
clf = model.named_steps['clf']

feature_names = tfidf.get_feature_names_out()
coefs = clf.coef_[0]

top_pos_idx = coefs.argsort()[-30:][::-1]
top_neg_idx = coefs.argsort()[:30]

top_pos_words = [(feature_names[i], coefs[i]) for i in top_pos_idx]
top_neg_words = [(feature_names[i], coefs[i]) for i in top_neg_idx]

print("Top words pushing to Top10 (class 1):")
for w, c in top_pos_words:
    print(w, c)

print("\nTop words pushing to non-Top10 (class 0):")
for w, c in top_neg_words:
    print(w, c)


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy: 0.6770428015564203
ROC-AUC: 0.7628366707097837
[[2703 1341]
 [ 319  777]]
              precision    recall  f1-score   support

           0       0.89      0.67      0.77      4044
           1       0.37      0.71      0.48      1096

    accuracy                           0.68      5140
   macro avg       0.63      0.69      0.62      5140
weighted avg       0.78      0.68      0.71      5140



/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/feature_extr

CV ROC-AUC mean: 0.5285180983632597 std: 0.06393180682845111
Top words pushing to Top10 (class 1):
lo lo 1.7528365591780688
i'm the 1.7020073733377497
come on 1.6754402497791467
you mean 1.5529395652602225
p 1.5241624700282563
roxanne 1.4949548587886028
i can 1.4833808630700045
want it 1.4045982406406894
yeah yeah 1.3503982945817938
work work 1.3355238465850026
down down 1.3037239664894336
can 1.2971509928605034
hands up 1.2889069118526348
g 1.2695383161525635
how you 1.266590186467021
boy 1.264568028957393
show you 1.26080398460805
smack that 1.2586926901930033
ooh woah 1.2504986782088952
let me 1.2435951299045584
just can't 1.2333012414730231
where where 1.2193623525878676
break break 1.2166757273082722
all night 1.2149930895089074
go head 1.214396446151351
together 1.2125307443288011
break your 1.2032889097600923
christmas 1.181517255130166
i like 1.173612755675762
wiggle 1.1658246515503885

Top words pushing to non-Top10 (class 0):
nigga -1.731869613963161
she -1.672039389225618
me

In [132]:
def make_period(year):
    if 2000 <= year <= 2009:
        return "2000s"
    elif 2010 <= year <= 2019:
        return "2010s"
    else:
        return "2020s"

merged['period'] = merged['year'].apply(make_period)
merged = merged.dropna(subset=['lyrics']).copy()
merged['is_top10'] = (merged['rank'] <= 10).astype(int)

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

period_results = {}

for period, df_p in merged.groupby('period'):
    X = df_p['lyrics'].astype(str)
    y = df_p['is_top10']

    model = Pipeline([
        ("tfidf", TfidfVectorizer(
            tokenizer=tokenize,
            min_df=5,
            max_df=0.7,
            ngram_range=(1,2)
        )),
        ("clf", LogisticRegression(
            max_iter=2000,
            class_weight="balanced"
        ))
    ])

    model.fit(X, y)

    tfidf = model.named_steps['tfidf']
    clf = model.named_steps['clf']
    feature_names = tfidf.get_feature_names_out()
    coefs = clf.coef_[0]

    top_pos_idx = coefs.argsort()[-50:][::-1]
    top_pos_words = [(feature_names[i], coefs[i]) for i in top_pos_idx]

    period_results[period] = {
        "model": model,
        "top_words": top_pos_words,
        "coefs": coefs,
        "features": feature_names
    }

    print("\n=== Period:", period, "===")
    for w, c in top_pos_words[:20]:
        print(w, c)


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



=== Period: 2000s ===
down down 1.526335115791331
mm 1.2739773542306772
my boo 1.254710848257856
boy 1.2158306009534525
smack that 1.1888180024911637
by 1.1883005874030823
stutter 1.1808083879024038
hot 1.1459998871969688
nah 1.135280921022944
low 1.1005909788719321
vs 1.0731590408992562
got 1.0456989877177483
lean back 1.0378719305757513
ladies 1.0376048345314635
you could 1.0083991071256664
nah nah 1.007571654657087
and over 0.9974697593034274
hey 0.9630636348954097
heartless 0.9573358687916164
boychi 0.9466221237992256


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



=== Period: 2010s ===
i'm the 1.6200046525511955
got it 1.5784170858716131
ha ha 1.495432003943323
wiggle 1.411328702393666
what do 1.4031051222928106
ha 1.3941517859731891
hands up 1.3916744895951398
i like 1.3867885149069037
work work 1.364195638764648
yeah yeah 1.3344781727156152
you mean 1.309027875874809
guy 1.2968847215072254
ta 1.272706746127735
ba 1.2697519368049033
wiggle wiggle 1.2489809029215881
where where 1.2271128577231651
can't stop 1.2017270233345931
let me 1.1936624195197438
come on 1.1927540240670569
said 1.1560965021274971


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



=== Period: 2020s ===
christmas 1.8026463343823103
lo lo 1.3865001176469909
it was 1.340023393458153
for 1.3320885767938433
jingle 1.296634138053305
roxanne 1.2639933663226561
do 1.192771307252589
ooh woah 1.1912219004376978
rockstar 1.1362667839101106
ding 1.1217138770895074
for you 1.1159403349246773
ooh 1.1127268555141492
ee 1.1086009259062848
stay 1.102768565345103
la la 1.031470307322233
la 1.022243620921164
run away 1.01605021971021
bruno 1.0155212464640095
ayy 0.9894984193131111
can love 0.9616842070573972


In [134]:
year_word_counts = {}
year_doc_counts = {}

for year, df_y in merged.dropna(subset=['lyrics']).groupby('year'):
    c = Counter()
    n_docs = 0
    for t in df_y['lyrics'].astype(str):
        tokens = set(tokenize(t))
        c.update(tokens)
        n_docs += 1
    year_word_counts[year] = c
    year_doc_counts[year] = n_docs

In [138]:
global_counter = Counter()
for c in year_word_counts.values():
    global_counter.update(c)

vocab = [w for w, cnt in global_counter.most_common(500)]
vocab

['you',
 'also',
 'might',
 'like',
 'the',
 'i',
 'and',
 'to',
 'me',
 'a',
 'in',
 'it',
 'my',
 'that',
 "i'm",
 'on',
 'know',
 'your',
 'so',
 'all',
 'of',
 "don't",
 'but',
 'just',
 'be',
 'up',
 'with',
 'for',
 'yeah',
 'when',
 'we',
 'this',
 "it's",
 'is',
 'get',
 'got',
 'no',
 'what',
 'oh',
 'now',
 'cause',
 'do',
 'out',
 'go',
 'if',
 'can',
 'love',
 'baby',
 'see',
 'make',
 'one',
 "can't",
 'at',
 "ain't",
 'down',
 'say',
 'never',
 'wanna',
 'time',
 'let',
 'back',
 "you're",
 'take',
 'from',
 'want',
 'come',
 'right',
 'need',
 'they',
 'how',
 'not',
 'way',
 'was',
 'girl',
 'tell',
 'have',
 'been',
 "that's",
 'she',
 'feel',
 "i'll",
 'then',
 'her',
 'could',
 'here',
 'keep',
 'off',
 'give',
 'ooh',
 'put',
 'good',
 'about',
 'look',
 'some',
 'more',
 'too',
 'think',
 'them',
 'life',
 'where',
 'night',
 'as',
 'man',
 'said',
 'uh',
 'are',
 'every',
 'by',
 'or',
 'even',
 'only',
 'had',
 'hey',
 'there',
 'still',
 'around',
 'why',
 'day'

In [139]:
rows = []
for year, c in year_word_counts.items():
    total = sum(c.values())
    for w in vocab:
        freq = c[w] / total if total > 0 else 0
        rows.append({"year": year, "token": w, "freq": freq})

trend_df = pd.DataFrame(rows)
trend_df

,year,token,freq
0,2000,you,0.006320
1,2000,also,0.006320
2,2000,might,0.006320
3,2000,like,0.006264
4,2000,the,0.006112
...,...,...,...
11995,2023,meet,0.000355
11996,2023,wake,0.000563
11997,2023,sometimes,0.000178
11998,2023,kind,0.000385


In [140]:
period_word_scores = {}

for period, df_p in merged.groupby('period'):
    years_in_p = df_p['year'].unique()
    sub = trend_df[trend_df['year'].isin(years_in_p)]
    mean_freq = sub.groupby('token')['freq'].mean()
    global_mean = trend_df.groupby('token')['freq'].mean()
    score = (mean_freq - global_mean).sort_values(ascending=False)
    period_word_scores[period] = score
    print("\n=== Period:", period, "top words by overuse ===")
    print(score.head(20))


=== Period: 2000s top words by overuse ===
token
world      0.000244
about      0.000239
right      0.000223
ya         0.000221
there      0.000214
why        0.000208
will       0.000203
boy        0.000202
girl       0.000201
there's    0.000201
or         0.000192
way        0.000190
after      0.000180
move       0.000178
y'all      0.000177
man        0.000169
hot        0.000166
next       0.000157
club       0.000150
before     0.000150
Name: freq, dtype: float64

=== Period: 2010s top words by overuse ===
token
we        0.000387
oh        0.000384
up        0.000380
my        0.000362
also      0.000347
might     0.000346
like      0.000339
you       0.000327
it        0.000309
on        0.000302
make      0.000298
the       0.000290
and       0.000282
i         0.000280
i'm       0.000276
woah      0.000268
me        0.000267
woo       0.000265
we're     0.000262
you're    0.000257
Name: freq, dtype: float64

=== Period: 2020s top words by overuse ===
token
fuck      0.0004

In [141]:
def make_decade(year):
    if 2000 <= year <= 2009:
        return "2000s"
    elif 2010 <= year <= 2019:
        return "2010s"
    else:
        return "2020s"

merged['decade'] = merged['year'].apply(make_decade)

In [142]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

data_dec = merged.dropna(subset=['lyrics']).copy()
X = data_dec['lyrics'].astype(str)
y = data_dec['decade']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

decade_model = Pipeline([
    ("tfidf", TfidfVectorizer(
        tokenizer=tokenize,
        min_df=10,
        max_df=0.7,
        ngram_range=(1,2)
    )),
    ("clf", LinearSVC())
])

decade_model.fit(X_train, y_train)
y_pred = decade_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[[1689  215   30]
 [ 141 2122   39]
 [  59  119  726]]
              precision    recall  f1-score   support

       2000s       0.89      0.87      0.88      1934
       2010s       0.86      0.92      0.89      2302
       2020s       0.91      0.80      0.85       904

    accuracy                           0.88      5140
   macro avg       0.89      0.87      0.88      5140
weighted avg       0.88      0.88      0.88      5140

